# Imports and reading data

In [184]:
import os
import pandas as pd
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt

DATA_FOLDER = "data"
PLOT_FOLDER = "plots"
CRM_PATH = os.path.join(DATA_FOLDER, "crm_data.csv")
WEB_TRACKING_PATH = os.path.join(DATA_FOLDER, "web_tracking.csv")

In [185]:
os.mkdir(PLOT_FOLDER)

In [116]:
crm_df = pd.read_csv(CRM_PATH)
web_tracking_df = pd.read_csv(WEB_TRACKING_PATH)

# EDA + Data validation

In [117]:
crm_df.head()

,user_id,lead_status,company,sign_up_date,industry,country
0,137,Converted,Company_0,2024-03-09,Healthcare,US
1,163,Converted,Company_1,2024-03-01,Finance,UK
2,103,Prospect,Company_2,2024-03-02,Healthcare,France
3,163,Lost,Company_3,2024-03-01,SaaS,US
4,171,Lost,Company_4,2024-03-07,Healthcare,Germany


In [118]:
crm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       60 non-null     int64 
 1   lead_status   60 non-null     object
 2   company       60 non-null     object
 3   sign_up_date  60 non-null     object
 4   industry      60 non-null     object
 5   country       60 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.9+ KB


In [119]:
crm_df.isna().sum()

user_id         0
lead_status     0
company         0
sign_up_date    0
industry        0
country         0
dtype: int64

In [120]:
crm_df.nunique()

user_id         47
lead_status      3
company         60
sign_up_date    11
industry         4
country          4
dtype: int64

**Warning:** user_id is not unique

In [121]:
crm_df["sign_up_date"] = pd.to_datetime(crm_df["sign_up_date"])

In [122]:
crm_df.sort_values("sign_up_date", inplace=True)

In [123]:
crm_df.drop_duplicates(subset='user_id', keep="last", inplace=True)

In [124]:
crm_df["sign_up_date"].describe()

count                               47
mean     2024-03-06 07:39:34.468084992
min                2024-03-01 00:00:00
25%                2024-03-03 00:00:00
50%                2024-03-07 00:00:00
75%                2024-03-09 12:00:00
max                2024-03-11 00:00:00
Name: sign_up_date, dtype: object

In [125]:
crm_df[crm_df.user_id.duplicated()] 

,user_id,lead_status,company,sign_up_date,industry,country


In [203]:
print(f'Conversion Rate = {(crm_df["lead_status"] == "Converted").sum() / len(crm_df)}')

Conversion Rate = 0.425531914893617


In [221]:
def plot_column_distribution(df, col_name, output_path=None):
    counts = df[col_name].value_counts()
    print(counts)

    fig, ax = plt.subplots()
    counts.plot(kind="bar", ax=ax)

    ax.set_xlabel(col_name)
    ax.set_ylabel("Count")
    ax.set_title(f"Distribution of {col_name}")

    ax.grid(axis="y", linestyle="--")

    plt.tight_layout()

    if output_path:
        plt.savefig(os.path.join(PLOT_FOLDER, output_path), format="png")
        plt.close()

    return ax

In [205]:
plot_column_distribution(crm_df, "lead_status", "lead_status_distribution.png")

lead_status
Converted    20
Lost         15
Prospect     12
Name: count, dtype: int64


<Axes: title={'center': 'Distribution of lead_status'}, xlabel='lead_status', ylabel='Count'>

In [206]:
plot_column_distribution(crm_df, "industry", "industry_distribution.png")

industry
Finance       15
E-commerce    14
SaaS          12
Healthcare     6
Name: count, dtype: int64


<Axes: title={'center': 'Distribution of industry'}, xlabel='industry', ylabel='Count'>

In [207]:
plot_column_distribution(crm_df, "country", "country_distribution.png")

country
US         23
UK         11
Germany    10
France      3
Name: count, dtype: int64


<Axes: title={'center': 'Distribution of country'}, xlabel='country', ylabel='Count'>

- There are three options for lead_status: ("converted", "lost", "prospect")
- lead_status = "converted" is the most likely to appear with 25 incides (42%)
- The largest groups of web activity:
    - Industry: Finance (22)
    - Country: US (27)
- The smallest groups of web activity:
    - Industry: Healthcare (9)
    - Country: France (7) 

In [131]:
def get_conversion_rate(values, prospects = True):
    num_converted = values[values == "Converted"].shape[0]
    if prospects:
        total = values.shape[0]
    else:
        total = values[values != "Prospect"].shape[0]
    percentage = num_converted / total * 100
    return percentage

In [192]:
def plot_conversion_rate_by_column(df, col_name, prospects=True, output_path=None):
    conv_rate = df.groupby(col_name)["lead_status"].apply(lambda x: get_conversion_rate(x, prospects))
    sorted_conv_rates = conv_rate.sort_values(ascending=False)
    print(sorted_conv_rates)

    ax = sorted_conv_rates.plot(kind="bar")
    ax.set_xlabel(col_name)
    ax.set_ylabel("Conversion Rate")
    ax.set_title(f"Conversion Rate by {col_name}")
    ax.grid(axis="y", linestyle="--")

    plt.tight_layout()

    if output_path:
        plt.savefig(os.path.join(PLOT_FOLDER, output_path), format="png")
        plt.close()
    
    return ax

In [191]:
plot_conversion_rate_by_column(crm_df, "country", output_path="conversion_rate_by_country.png")

country
US         56.521739
UK         36.363636
France     33.333333
Germany    20.000000
Name: lead_status, dtype: float64
save


<Axes: title={'center': 'Conversion Rate by country'}, xlabel='country', ylabel='Conversion Rate'>

In [198]:
plot_conversion_rate_by_column(crm_df, "industry", output_path="conversion_rate_by_industry.png")

industry
Healthcare    83.333333
SaaS          41.666667
E-commerce    35.714286
Finance       33.333333
Name: lead_status, dtype: float64


<Axes: title={'center': 'Conversion Rate by industry'}, xlabel='industry', ylabel='Conversion Rate'>

- The US has highest conversion rate (59%) in contrast to France with the lowest (14%)
- Healthcare has highest conversion rate (55%) in contrast to E-commerce (36%)
- **NOTE:** This conversion rate calculation also includes lead_status = "prospects"

In [199]:
plot_conversion_rate_by_column(crm_df, "industry", False, "conversion_rate_by_industry_no_prospects.png")

industry
Healthcare    83.333333
SaaS          62.500000
E-commerce    50.000000
Finance       45.454545
Name: lead_status, dtype: float64


<Axes: title={'center': 'Conversion Rate by industry'}, xlabel='industry', ylabel='Conversion Rate'>

In [196]:
plot_conversion_rate_by_column(crm_df, "country",  False, "conversion_rate_by_country_no_prospects.png")

country
US         68.421053
UK         57.142857
France     33.333333
Germany    33.333333
Name: lead_status, dtype: float64


<Axes: title={'center': 'Conversion Rate by country'}, xlabel='country', ylabel='Conversion Rate'>

- findings change drastically by not including lead_status = "Prospect"
    - Industry: Conversion rate differences are less between industries (ranges from 51% (E-commerce) to 71% (Healthcare)
    - Country: The lowest country now is Germany with 25% and US with increased conversionrate of 69%

# Web Tracking Data

In [137]:
web_tracking_df.columns

Index(['user_id', 'session_id', 'timestamp', 'page', 'time_spent_sec',
       'utm_source', 'utm_medium', 'device_type'],
      dtype='object')

In [138]:
web_tracking_df.head()

,user_id,session_id,timestamp,page,time_spent_sec,utm_source,utm_medium,device_type
0,151,s000,2024-03-07 12:55:00,/pricing,54,newsletter,referral,mobile
1,192,s001,2024-03-05 14:21:00,/landing,71,linkedin,email,desktop
2,114,s002,2024-03-01 14:25:00,/contact,143,facebook,cpc,desktop
3,171,s003,2024-03-06 16:56:00,/contact,293,newsletter,email,desktop
4,160,s004,2024-03-01 21:41:00,/contact,37,linkedin,organic,tablet


In [229]:
web_tracking_df.nunique()

user_id            61
session_id        100
timestamp          99
page                4
time_spent_sec     74
utm_source          4
utm_medium          4
device_type         3
dtype: int64

In [139]:
web_tracking_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   user_id         100 non-null    int64 
 1   session_id      100 non-null    object
 2   timestamp       100 non-null    object
 3   page            100 non-null    object
 4   time_spent_sec  100 non-null    int64 
 5   utm_source      100 non-null    object
 6   utm_medium      100 non-null    object
 7   device_type     100 non-null    object
dtypes: int64(2), object(6)
memory usage: 6.4+ KB


In [140]:
web_tracking_df["timestamp"] = pd.to_datetime(web_tracking_df["timestamp"])

In [141]:
web_tracking_df.time_spent_sec.describe()

count    100.000000
mean     163.750000
std       71.941106
min       11.000000
25%      121.750000
50%      156.500000
75%      227.500000
max      295.000000
Name: time_spent_sec, dtype: float64

In [142]:
web_tracking_df.timestamp.describe()

count                              100
mean     2024-03-04 12:31:25.199999744
min                2024-03-01 01:18:00
25%                2024-03-02 18:32:15
50%                2024-03-04 16:09:30
75%                2024-03-06 04:45:15
max                2024-03-07 21:13:00
Name: timestamp, dtype: object

**NOTE:** Web tracking until 03-07, but users signed up until 03-10

In [226]:
def plot_time_spent_hist(df, time_col='time_spent_sec', output_path=None):
    ax = df[time_col].plot(kind="hist")

    ax.set_xlabel('Time Spent (seconds)')
    ax.set_ylabel('Frequency')
    ax.set_title('Distribution of Time Spent')

    if output_path:
        plt.savefig(os.path.join(PLOT_FOLDER, output_path), format="png")
        plt.close()

    return ax


In [227]:
plot_time_spent_hist(df=web_tracking_df, output_path='hist.png')

<Axes: title={'center': 'Distribution of Time Spent'}, xlabel='Time Spent (seconds)', ylabel='Frequency'>

In [213]:
web_tracking_df.time_spent_sec.describe()

count    100.000000
mean     163.750000
std       71.941106
min       11.000000
25%      121.750000
50%      156.500000
75%      227.500000
max      295.000000
Name: time_spent_sec, dtype: float64

In [144]:
web_tracking_df.nunique()

user_id            61
session_id        100
timestamp          99
page                4
time_spent_sec     74
utm_source          4
utm_medium          4
device_type         3
dtype: int64

**Warning:** Some users are missing in the crm

In [166]:
missing_users = set(web_tracking_df["user_id"]) - set(crm_df["user_id"])
len(missing_users)

14

**NOTE:** user_id {101, 105, 113, 129, 133, 135, 139, 148, 153, 172, 175, 179, 182, 186} have web tracking data but are missing for crm

In [146]:
web_tracking_df.isna().sum()

user_id           0
session_id        0
timestamp         0
page              0
time_spent_sec    0
utm_source        0
utm_medium        0
device_type       0
dtype: int64

In [147]:
web_tracking_df.sort_values("timestamp", inplace = True)

In [148]:
web_tracking_df[web_tracking_df["user_id"] == 161]

,user_id,session_id,timestamp,page,time_spent_sec,utm_source,utm_medium,device_type
37,161,s037,2024-03-01 06:29:00,/contact,11,facebook,referral,tablet
40,161,s040,2024-03-03 18:03:00,/contact,63,facebook,email,tablet
38,161,s038,2024-03-04 18:03:00,/pricing,139,facebook,cpc,tablet
93,161,s093,2024-03-06 17:50:00,/features,63,google,referral,mobile
84,161,s084,2024-03-07 03:22:00,/features,46,google,cpc,tablet


In [149]:
nb_of_pages_visited = web_tracking_df.user_id.value_counts()
nb_of_pages_visited_df = nb_of_pages_visited.to_frame().reset_index()
nb_of_pages_visited_df.rename(columns = {"count":"numb_pages"}, inplace = True)

In [150]:
nb_of_pages_visited

user_id
161    5
159    4
101    4
120    3
114    3
      ..
149    1
154    1
138    1
105    1
173    1
Name: count, Length: 61, dtype: int64

In [151]:
def get_last_page(x):
    last_page = x.to_list()[-1]
    return last_page

In [152]:
last_page = web_tracking_df.groupby('user_id').page.apply(get_last_page)

In [153]:
last_page_df = last_page.to_frame().reset_index()
last_page_df.rename(columns = {"page":"last_page"}, inplace=True)

In [154]:
def get_first_page(x):
    first_page = x.to_list()[0]
    return first_page

In [155]:
first_page = web_tracking_df.groupby('user_id').page.apply(get_first_page)

In [156]:
first_page_df = first_page.to_frame().reset_index()
first_page_df.rename(columns = {"page":"first_page"}, inplace=True)

In [157]:
merged_df = crm_df.merge(last_page_df, on = "user_id")\
            .merge(first_page_df, on = "user_id")\
            .merge(nb_of_pages_visited_df, on = "user_id")

In [158]:
merged_df

,user_id,lead_status,company,sign_up_date,industry,country,last_page,first_page,numb_pages
0,184,Prospect,Company_18,2024-03-01,Finance,US,/features,/features,1
1,163,Lost,Company_3,2024-03-01,SaaS,US,/features,/pricing,2
2,199,Converted,Company_32,2024-03-01,Healthcare,US,/landing,/features,2
3,151,Converted,Company_5,2024-03-01,Finance,US,/pricing,/pricing,1
4,191,Prospect,Company_42,2024-03-01,SaaS,Germany,/pricing,/contact,2
5,162,Lost,Company_12,2024-03-01,Finance,Germany,/contact,/contact,1
6,106,Prospect,Company_49,2024-03-01,Finance,Germany,/features,/features,1
7,107,Prospect,Company_19,2024-03-01,E-commerce,US,/features,/features,1
8,154,Lost,Company_11,2024-03-01,Finance,US,/features,/features,1
9,141,Prospect,Company_6,2024-03-02,E-commerce,Germany,/landing,/landing,1


In [217]:
plot_conversion_rate_by_column(merged_df, "last_page", False,"conversion_rate_by_last_page.png")

last_page
/landing     70.000000
/pricing     66.666667
/features    50.000000
/contact     40.000000
Name: lead_status, dtype: float64


<Axes: title={'center': 'Conversion Rate by last_page'}, xlabel='last_page', ylabel='Conversion Rate'>

- if a user ends on contact page he/she does not converse in 3 out of 4 cases
- if a user ends on pricing page he/she converse in 54% of the cases

In [218]:
plot_conversion_rate_by_column(merged_df, "first_page", False,"conversion_rate_by_first_page.png")

first_page
/landing     70.000000
/features    66.666667
/pricing     57.142857
/contact     16.666667
Name: lead_status, dtype: float64


<Axes: title={'center': 'Conversion Rate by first_page'}, xlabel='first_page', ylabel='Conversion Rate'>

In [228]:
merged_df.numb_pages.value_counts().to_clipboard()

In [220]:
plot_conversion_rate_by_column(merged_df, "numb_pages", False,"conversion_rate_by_numb_page.png")

numb_pages
4    100.000000
5    100.000000
1     57.894737
2     50.000000
3     50.000000
Name: lead_status, dtype: float64


<Axes: title={'center': 'Conversion Rate by numb_pages'}, xlabel='numb_pages', ylabel='Conversion Rate'>

In [163]:
# pandas profiling
crm_profile = ProfileReport(crm_df, 
                            title="CRM Data Profiling Report", 
                            explorative=True)
web_tracking_profile = ProfileReport(web_tracking_df, 
                                     title="Web Tracking Data Profiling Report", 
                                     explorative=True)
crm_profile.to_file("crm_data_profile.html")
web_tracking_profile.to_file("web_tracking_data_profile.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 6/6 [00:00<00:00, 38.54it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 8/8 [00:00<00:00, 125.98it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]